## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
# place data in a folder called /content/ or change file path:

# Preprocess the data
df = pd.read_csv('RRCA_baseflow.csv')
#display(df.head(5))
# df.describe()
df['Date'] -= 693963
df['Date'] = pd.to_datetime(df['Date'], origin='1900-01-01', unit='D')

## Analysis

In [ ]:
# Add columns Year and Month
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

# Add columns month_shift and days_since_rec_start
df["month_shifted"] =  abs( df["Month"]-7)
df["days_since_rec_start"] = (6-df["month_shifted"]) * 30

In [ ]:
# Group data by year
grouped = df.groupby('Year').mean()

# Plot line graph for each year
plt.scatter(grouped.index, grouped['Observed'])
plt.plot(grouped.index, grouped['Observed'])

# Add labels and title
plt.xlabel('Year')
plt.ylabel('Mean Observed')
plt.title('Mean Observed Data Over Time')

# Show plot
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Data post 1950 was way larger than current data

In [ ]:
plt.figure()
sns.lineplot(data=df, x="Date", y="Observed")
plt.title("Observed over Time")

plt.figure()
sns.lineplot(data=df, x="Date", y="Irrigation_pumping")
plt.title("Irrigation_pumping over Time")

In [ ]:
df = df[df['Date'].dt.year >= 1950]

Observed Data Over Time from April to October(summer) and November to March(winter)

In [ ]:
# Divide data into two periods April to October and November to March
df_apr_oct = df[(df['Month'] >= 4) & (df['Month'] <= 10)]
df_nov_mar = df[(df['Month'] >= 11) | (df['Month'] <= 3)]

# Group data by year for each period and calculate the mean
grouped_apr_oct = df_apr_oct.groupby(df_apr_oct['Date'].dt.year).mean()
grouped_nov_mar = df_nov_mar.groupby(df_nov_mar['Date'].dt.year).mean()

# Determine common y-axis limits
y_min = min(grouped_apr_oct['Observed'].min(), grouped_nov_mar['Observed'].min())
y_max = max(grouped_apr_oct['Observed'].max(), grouped_nov_mar['Observed'].max())

# Plot line graph for each period
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(grouped_apr_oct.index, grouped_apr_oct['Observed'])
plt.xlabel('Year')
plt.ylabel('Mean Observed')
plt.title('April to October: Mean Observed Data Over Time')
plt.ylim(y_min, y_max)

plt.subplot(1, 2, 2)
plt.plot(grouped_nov_mar.index, grouped_nov_mar['Observed'])
plt.xlabel('Year')
plt.ylabel('Mean Observed')
plt.title('November to March: Mean Observed Data Over Time')
plt.ylim(y_min, y_max)

plt.tight_layout()
plt.show()

Data observed for Each Season spring, summer, fall, winter

In [ ]:
# Define season
seasons = {
    'spring': (3, 5),
    'summer': (6, 9),
    'fall': (10, 11),
    'winter': (12, 2)
}

# List of columns
columns_to_plot = ['Observed', 'Evapotranspiration', 'Precipitation', 'Irrigation_pumping']

# Loop through each column
for column in columns_to_plot:
    plt.figure(figsize=(16, 8))
    for season, (start_month, end_month) in seasons.items():
        # Extract data for the current season
        season_data = df[(df['Month'] >= start_month) & (df['Month'] <= end_month)]

        # Group data by year and compute the mean for each year
        season_data = season_data.groupby('Year')[column].mean()

        # Plot line graph for the current season
        plt.plot(season_data.index, season_data.values, label=season)

    # Add labels and legend
    plt.xlabel('Year')
    plt.ylabel(column)
    plt.title(f'{column} Data for Each Season')
    plt.legend()
    plt.show()

In [ ]:
unique_segment_ids = df['Segment_id'].nunique()
print("Count of unique Segment IDs:", unique_segment_ids)

In [ ]:
# Plot a line for each segment
plt.figure(figsize=(12, 8))

# Get unique segment IDs
segment_year_median = df.groupby(['Segment_id', 'Year'])['Observed','Evapotranspiration','Precipitation'].median().reset_index()
unique_segment_ids = segment_year_median['Segment_id'].unique()

# Determine the number of rows and columns for subplots
num_rows = (len(unique_segment_ids) + 1) // 2  # Add 1 and use floor division
num_cols = 2

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(16, 12))

# Flatten axes if necessary
if len(unique_segment_ids) > 1:
    axes = axes.flatten()

# Iterate over segment IDs and plot on subplots
for i, segment_id in enumerate(unique_segment_ids):
    segment_data = segment_year_median[segment_year_median['Segment_id'] == segment_id]
    ax = axes[i] if len(unique_segment_ids) > 1 else axes
    ax.plot(segment_data['Year'], segment_data['Observed'])
    ax.plot(segment_data['Year'], segment_data['Evapotranspiration'])
    ax.plot(segment_data['Year'], segment_data['Precipitation'])
    ax.set_title(f'Segment {segment_id}')
    ax.set_xlabel('Year')
    ax.set_ylabel('Observed')

# Adjust layout
plt.show()


Data observed over months 

In [ ]:
sns.lineplot(data=df, x="Month", y="Evapotranspiration")
plt.title("Evapotranspiration over Months")

plt.figure()
sns.lineplot(data=df, x="Month", y="Precipitation")
plt.title("Precipitation over Months")

plt.figure()
sns.lineplot(data=df, x="Month", y="Irrigation_pumping")
plt.title("Irrigation_pumping over Months")

In [ ]:
sns.lineplot(data=df, x="days_since_rec_start", y="Evapotranspiration")
plt.title("Evapotranspiration over Baseflow Recession")

plt.figure()
sns.lineplot(data=df, x="days_since_rec_start", y="Precipitation")
plt.title("Precipitation over Baseflow Recession")

plt.figure()
sns.lineplot(data=df, x="days_since_rec_start", y="Irrigation_pumping")
plt.title("Irrigation_pumping over Baseflow Recession")

## Linear Regression

### Using "formula notation"

In [ ]:

lm = smf.ols(formula='Observed ~ Segment_id	+ x + y + Month + Year +  Evapotranspiration + Precipitation + Irrigation_pumping', data=df).fit()

# print a summary of the fitted model
print(lm.summary())

### LinearRegression

In [ ]:
df.columns

With One-hot encode

In [ ]:
feature_cols = ['Segment_id', 'x', 'y', 'Evapotranspiration', 'Precipitation',
       'Irrigation_pumping', 'Year', 'Month']

# X = df[feature_cols]
y = df['Observed']

# One-hot encode categorical variables
X = pd.get_dummies(df[feature_cols], columns=['Segment_id', 'x', 'y'], drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model
lm = LinearRegression()
lm.fit(X_train, y_train)

# Make predictions on the test data
y_pred = lm.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R2): {r2}')


without One-hot encode

In [ ]:
feature_cols = ['Segment_id', 'x', 'y', 'Evapotranspiration', 'Precipitation',
       'Irrigation_pumping', 'Year', 'Month']

X = df[feature_cols]
y = df['Observed']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model
lm = LinearRegression()
lm.fit(X_train, y_train)

# Make predictions on the test data
y_pred = lm.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R2): {r2}')
